In [71]:
import os
import pandas as pd
import re

In [72]:
mobility_path = '../data/raw/'
processed_path = '../data/processed/'
raw_path = '../data/raw/'


In [73]:
mobility = pd.read_csv(mobility_path+'Global_Mobility_Report.csv', low_memory=False)
processed = pd.read_csv(processed_path + 'time_series.csv')
zips = pd.read_csv(raw_path + 'uszips.csv')

In [74]:
# clean USA data
USA = mobility.loc[mobility['country_region'] == 'United States'].reset_index()
to_drop = ['index', 'country_region_code', 'country_region']
USA = USA.drop(to_drop, axis=1)
USA.insert(1, 'county', USA['sub_region_2'].str.lower())
USA = USA.drop(['sub_region_2'],axis=1)
USA = USA.loc[~pd.isna(USA['county'])]
USA['county'] = USA['county'].map(lambda x: re.sub(' county| city','', x)).str.lower()
USA.insert(1, 'state', USA['sub_region_1'].str.lower())
USA = USA.drop(['sub_region_1'],axis=1)

In [75]:
USA

,state,county,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
114,alabama,autauga,2020-02-15,5.0,7.0,NaN,NaN,-4.0,NaN
115,alabama,autauga,2020-02-16,0.0,1.0,-23.0,NaN,-4.0,NaN
116,alabama,autauga,2020-02-17,8.0,0.0,NaN,NaN,-27.0,5.0
117,alabama,autauga,2020-02-18,-2.0,0.0,NaN,NaN,2.0,0.0
118,alabama,autauga,2020-02-19,-2.0,0.0,NaN,NaN,2.0,0.0
...,...,...,...,...,...,...,...,...,...
155055,wyoming,weston,2020-04-06,NaN,NaN,NaN,NaN,-39.0,NaN
155056,wyoming,weston,2020-04-07,NaN,NaN,NaN,NaN,-37.0,NaN
155057,wyoming,weston,2020-04-08,NaN,NaN,NaN,NaN,-36.0,NaN
155058,wyoming,weston,2020-04-09,NaN,NaN,NaN,NaN,-31.0,NaN


In [76]:
processed['county'] = processed['county'].str.lower()
processed['state'] = processed['state'].str.lower()

In [77]:
processed

,date,county,state,fips,cases,deaths,cldCvrMin,cldCvrAvg,cldCvrMax,dewPtMin,...,windSpd80mAvg,windSpd80mMax,windSpd100mMin,windSpd100mAvg,windSpd100mMax,wetBulbMin,wetBulbAvg,wetBulbMax,first_date,days_since_10_cases
0,2020-01-21,snohomish,washington,53061,1,0,7.0,71.0,100.0,37.4,...,18.9,22.1,14.0,18.0,20.7,41.6,42.7,44.1,2020-03-05,-44
1,2020-01-22,snohomish,washington,53061,1,0,31.0,74.0,100.0,39.9,...,13.6,21.2,10.0,13.2,20.5,41.4,42.4,44.6,2020-03-05,-43
2,2020-01-23,snohomish,washington,53061,1,0,75.0,93.0,100.0,44.3,...,15.5,21.4,10.6,15.4,19.4,44.9,48.6,50.7,2020-03-05,-42
3,2020-01-24,snohomish,washington,53061,1,0,15.0,57.0,95.0,43.9,...,15.3,23.2,9.6,14.2,20.7,44.7,46.7,48.3,2020-03-05,-41
4,2020-01-25,snohomish,washington,53061,1,0,17.0,72.0,100.0,41.4,...,8.9,15.3,4.2,8.1,12.7,42.5,44.5,48.0,2020-03-05,-40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54946,2020-04-15,saline,nebraska,31151,1,0,0.0,1.0,12.0,17.4,...,8.2,21.3,2.7,7.3,16.5,20.0,33.8,42.8,2020-04-21,-6
54947,2020-04-16,saline,nebraska,31151,3,0,45.0,88.0,100.0,22.7,...,21.4,25.0,16.1,20.0,23.3,30.0,32.3,34.5,2020-04-21,-5
54948,2020-04-17,saline,nebraska,31151,4,0,0.0,35.0,100.0,23.9,...,13.2,29.4,6.9,12.4,27.7,24.8,33.5,40.9,2020-04-21,-4
54949,2020-04-18,saline,nebraska,31151,6,0,0.0,9.0,69.0,30.1,...,23.8,28.3,7.0,24.1,31.6,33.1,45.3,55.4,2020-04-21,-3


In [78]:
# take only subset of counties according to proccessed data
counties = pd.unique(processed['county'].str.lower().map(lambda x: re.sub(' city','', x)))
USA = USA.loc[USA['county'].str.lower().isin(counties)]

In [79]:
# difference in counties
difference = set(pd.unique(counties)) - set(USA['county'].str.lower().unique())

In [80]:
# Cut time
# USA = USA.loc[USA['date']<'2020-04-23']

In [81]:
# deal with duplicates
columns = set(USA.columns) - set(['state', 'county', 'date'])
agg = {key:'mean' for key in columns}
USA_grouped = USA.groupby(['state', 'county', 'date']).aggregate(agg)

In [82]:
df = pd.merge(processed,USA_grouped, on=['date','county', 'state'], how='left')

In [84]:
df.to_csv(processed_path + 'time_series_mobility.csv')
# df = pd.read_csv(processed_path + 'time_series_mobility.csv')